In [43]:
import requests # Импортируем библиотеку requests для запросов
from bs4 import BeautifulSoup # Импортируем библиотеку BeautifulSoup для парсинга
import bleach  # Импортируем библиотеку bleach для очистки текста от тэгов
import pandas as pd
import os.path

In [44]:
columns_list=['recipe_id', 'cuisine', 'meal_mode', 'recipe_name', 
            'portions', 'calories', 'proteinContent',
            'fatContent', 'carbohydrateContent', 'ingr_name', 
            'ingr_qwn', 'measure', 'quantity', 'text']

In [45]:
# Считываем таблицу ссылок рецептов из файла.
url_df = pd.read_csv('./all_url_df.csv')
len(url_df)

9428

In [46]:
# Если файлы таблиц существуют, считываем их
if os.path.isfile('./ingr_df.csv'):
    # Считываем таблицу ингредиентов из файла.
    ingr_df = pd.read_csv('./ingr_df.csv')
else:
    # Создаем таблицу таблицу ингредиентов, если файла с ней нет.
    ingr_df = pd.DataFrame(columns=['ingr_id', 'ingr_name'])
    
if os.path.isfile('./recipe_df.csv'):
    # Считываем таблицу рецептов из файла.
    recipe_df = pd.read_csv('./recipe_df.csv')
else:
    # Создаем таблицу таблицу рецептов, если файла с ней нет.
    recipe_df = pd.DataFrame(columns=columns_list)

In [47]:
def add_ingr(ingr_name, ingr_df):
    """
    Функция добавляет ингредиент с уникальным именем в таблицу ингредиентов.

        Args:
            ingr_name (string): наименование ингредиента
            ingr_df (dataframe): таблица ингредиентов.
 
        Returns:
            ingr_df (dataframe): таблица ингредиентов с добавленным уникальным ингредиентом.    
    """
    # dataframe текущего ингредиента
    curr_ingr = pd.DataFrame(columns=['ingr_id', 'ingr_name'], index=range(1))
    curr_ingr.ingr_name = ingr_name # наименование текущего ингредиента
    curr_ingr.ingr_id = ingr_df.ingr_id.count() # id текущего ингредиента
    # присоединение текущего ингредиента к исходному dataframe 
    ingr_df = pd.concat([ingr_df, curr_ingr], ignore_index=True)
    # удаление ингредиента, если такое наименование уже было в исходном dataframe  
    ingr_df.drop_duplicates(subset=['ingr_name'], inplace=True)
    return ingr_df

In [48]:
for i in range(0,100): # Выбор ссылок для частичного парсинга из всех 9428 ссылок
    url = url_df.url[i]
    try:
        # Выполняем GET-запрос, содержимое ответа присваивается переменной response
        response = requests.get(url, headers={
        'User-Agent': 
        'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'})
        # Создаём объект BeautifulSoup, указывая html-парсер
        page = BeautifulSoup(response.text, 'html.parser') 
    except:
        pass
    # Кухня
    try:
        cuisine = page.title.text.split(" – ")[1].split(":")[0]
    except:
        cuisine = None
    # meal_mode    
    try:
        meal_mode = page.title.text.split(": ")[1].split(".")[0]
    except:
        meal_mode = None
    # Название рецепта
    try:
        recipe_name = page.find('h1').text.replace('\xa0',' ')
    except:
        recipe_name = None
    # portions
    try:
        portions = page.find(itemprop="recipeYield")
        portions = int(bleach.clean(str(portions), tags=[], strip=True))
    except:
        portions = None
    # calories
    try:
        calories = page.find(itemprop="calories")
        calories= int(bleach.clean(str(calories), tags=[], strip=True))
    except:
        calories = None
    # proteinContent
    try:
        proteinContent = page.find(itemprop="proteinContent")
        proteinContent= int(bleach.clean(str(proteinContent), tags=[], strip=True))
    except:
        proteinContent = None
    # fatContent
    try:
        itemProp="fatContent"
        fatContent = page.find(itemprop="fatContent")
        fatContent= int(bleach.clean(str(fatContent), tags=[], strip=True))
    except:
        fatContent = None
    # carbohydrateContent
    try:
        carbohydrateContent = page.find(itemprop="carbohydrateContent")
        carbohydrateContent= int(bleach.clean(str(carbohydrateContent), tags=[], strip=True))
    except:
        carbohydrateContent = None
    # ingr_names_list
    ingr_names = page.find_all(itemprop="recipeIngredient")
    ingr_names_list = []
    for ingr_name in ingr_names:
        ingr_name =  bleach.clean(str(ingr_name), tags=[], strip=True)
        ingr_names_list.append(ingr_name)
    # iingr_qwn_list
    ingr_qnts = page.find_all(class_="emotion-15im4d2")
    iingr_qwn_list = []
    for ingr_qwn in ingr_qnts:
        ingr_qwn =  bleach.clean(str(ingr_qwn), tags=[], strip=True)
        iingr_qwn_list.append(ingr_qwn)
    # Словарь ингредиентов текущего рецепта
    ingr_dict = {}
    if len(ingr_names_list) == len(iingr_qwn_list):
        for i in range(len(ingr_names_list)):
            ingr_dict[ingr_names_list[i]] = iingr_qwn_list[i]
            # Добавление ингредиентов текущего рецепта в таблицу ингредиентов
            ingr_df = add_ingr(ingr_names_list[i], ingr_df)
    # Текст рецепта
    rtl = page.find_all(itemprop="recipeInstructions")
    recipe_text_list = []
    for i in range(len(rtl)):
        recipe_step = rtl[i]
        recipe_step = bleach.clean(str(recipe_step), tags=[], strip=True)
        recipe_step = recipe_step.replace('\n','').replace('\xa0',' ')
        if i>=9:
            a = 2
        else: a = 1
        recipe_step = ("").join(recipe_step.split("Шпаргалка")[0])
        recipe_step = ("").join(recipe_step.split("Инструмент")[0])[a:]
        recipe_text_list.append(recipe_step)
    recipe_text = (" ").join(recipe_text_list)    

    # Таблица текущего рецепта
    recipe = pd.DataFrame(columns=columns_list)
    recipe.ingr_name = pd.Series(ingr_dict.keys())
    recipe.ingr_qwn = pd.Series(ingr_dict.values())
    recipe.recipe_name = recipe_name
    recipe.portions = portions
    recipe.calories = calories
    recipe.proteinContent = proteinContent
    recipe.fatContent = fatContent
    recipe.carbohydrateContent = carbohydrateContent
    recipe.text = recipe_text
    recipe.cuisine = cuisine
    recipe.meal_mode = meal_mode
    if len(recipe_df) == 0: #если это первый рецепт в таблице
        recipe.recipe_id = 0
    else:
        recipe.recipe_id = recipe_df.recipe_id.iloc[-1]+1

    # Присоединение текущего рецепта к таблице рецептов
    recipe_df = pd.concat([recipe_df, recipe], ignore_index=True)
# удаление дубликатов рецептов 
recipe_df.drop_duplicates(inplace=True, subset=['recipe_name', 'ingr_name'])

In [49]:
# Запись в файлы
recipe_df.to_csv('./recipe_df.csv', index=False)
ingr_df.to_csv('./ingr_df.csv', index=False)